In [ ]:
# Install required libraries
!pip install tensorflow
!pip install sentencepiece
!pip install pandas
!pip install numpy

# Verify TensorFlow and GPU
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# Enable memory growth for GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

TensorFlow version: 2.18.0
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import all necessary libraries
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import gc
import os

# Create necessary directories in Google Drive
os.makedirs('/content/drive/MyDrive/seq2seq_models', exist_ok=True)
os.makedirs('/content/drive/MyDrive/seq2seq_data', exist_ok=True)

Mounted at /content/drive


In [ ]:
# Load large dataset in chunks
def load_parallel_corpus(amh_file, eng_file, chunk_size=10000):  # Increased chunk size for GPU
    amharic_sentences = []
    english_sentences = []

    with open(amh_file, 'r', encoding='utf-8') as f_amh, open(eng_file, 'r', encoding='utf-8') as f_eng:
        while True:
            amh_chunk = [next(f_amh, '').strip() for _ in range(chunk_size)]
            eng_chunk = [next(f_eng, '').strip() for _ in range(chunk_size)]

            if not amh_chunk[0] or not eng_chunk[0]:  # End of file
                break

            amharic_sentences.extend(amh_chunk)
            english_sentences.extend(eng_chunk)

            # Clear memory
            gc.collect()

    return amharic_sentences, english_sentences

def tokenize_in_chunks(sentences, tokenizer, chunk_size=1000):  # Increased chunk size for GPU
    encoded = []
    for i in range(0, len(sentences), chunk_size):
        chunk = sentences[i:i + chunk_size]
        encoded.extend([tokenizer.encode_as_ids(sent) for sent in chunk])
        gc.collect()
    return encoded

In [ ]:
# Move the uploaded files to Google Drive
!mv amh.txt /content/drive/MyDrive/seq2seq_data/
!mv eng.txt /content/drive/MyDrive/seq2seq_data/

# Load data
print("Loading data...")
amharic_sentences, english_sentences = load_parallel_corpus(
    "/content/drive/MyDrive/seq2seq_data/amh.txt",
    "/content/drive/MyDrive/seq2seq_data/eng.txt"
)

# Train SentencePiece tokenizers
print("Training tokenizers...")
for lang, sentences in [("amh", amharic_sentences), ("eng", english_sentences)]:
    with open(f"/content/drive/MyDrive/seq2seq_data/{lang}.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(sentences))
    spm.SentencePieceTrainer.train(
        input=f"/content/drive/MyDrive/seq2seq_data/{lang}.txt",
        model_prefix=f"/content/drive/MyDrive/seq2seq_models/tokenizer_{lang}",
        vocab_size=4000,
        model_type="unigram",
        character_coverage=1.0,
        pad_id=0,
        unk_id=1,
        bos_id=2,
        eos_id=3
    )

Loading data...
Training tokenizers...


In [ ]:
# Load tokenizers
print("Loading tokenizers...")
sp_amh = spm.SentencePieceProcessor(model_file="/content/drive/MyDrive/seq2seq_models/tokenizer_amh.model")
sp_eng = spm.SentencePieceProcessor(model_file="/content/drive/MyDrive/seq2seq_models/tokenizer_eng.model")

print("Tokenizing data...")
amh_encoded = tokenize_in_chunks(amharic_sentences, sp_amh)
eng_encoded = tokenize_in_chunks(english_sentences, sp_eng)

# Clear memory
del amharic_sentences, english_sentences
gc.collect()

# Pad sequences
print("Padding sequences...")
max_len_amh = max(len(x) for x in amh_encoded)
max_len_eng = max(len(x) for x in eng_encoded)
amh_padded = tf.keras.preprocessing.sequence.pad_sequences(amh_encoded, maxlen=max_len_amh, padding="post")
eng_padded = tf.keras.preprocessing.sequence.pad_sequences(eng_encoded, maxlen=max_len_eng, padding="post")

# Clear memory
del amh_encoded, eng_encoded
gc.collect()

# Split data into train and validation
print("Splitting data...")
val_split = 0.1
val_size = int(len(amh_padded) * val_split)
train_amh = amh_padded[:-val_size]
train_eng = eng_padded[:-val_size]
val_amh = amh_padded[-val_size:]
val_eng = eng_padded[-val_size:]

# Clear memory
del amh_padded, eng_padded
gc.collect()

Loading tokenizers...
Tokenizing data...
Padding sequences...
Splitting data...


0

In [ ]:
# Model parameters - increased for GPU
embedding_dim = 128  # Increased for GPU
lstm_units = 256    # Increased for GPU

print("Building model...")
# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=4000, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=4000, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
decoder_dense = Dense(4000, activation="softmax")(decoder_outputs)

# Create model
model = Model([encoder_inputs, decoder_inputs], decoder_dense)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

Building model...


In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        "/content/drive/MyDrive/seq2seq_models/seq2seq_model.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

# Train model with larger batch size for GPU
print("Starting training...")
history = model.fit(
    [train_amh, train_eng[:, :-1]],
    tf.expand_dims(train_eng[:, 1:], -1),
    validation_data=(
        [val_amh, val_eng[:, :-1]],
        tf.expand_dims(val_eng[:, 1:], -1)
    ),
    batch_size=64,  # Increased batch size for GPU
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

Starting training...
Epoch 1/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.9312 - loss: 0.4715

282/282 ━━━━━━━━━━━━━━━━━━━━ 45s 159ms/step - accuracy: 0.9312 - loss: 0.4715 - val_accuracy: 1.0000 - val_loss: 9.2231e-04
Epoch 2/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.9332 - loss: 0.4529

282/282 ━━━━━━━━━━━━━━━━━━━━ 85s 170ms/step - accuracy: 0.9332 - loss: 0.4528 - val_accuracy: 1.0000 - val_loss: 4.9197e-04
Epoch 3/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.9339 - loss: 0.4339

282/282 ━━━━━━━━━━━━━━━━━━━━ 83s 172ms/step - accuracy: 0.9339 - loss: 0.4339 - val_accuracy: 1.0000 - val_loss: 3.0912e-04
Epoch 4/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.9365 - loss: 0.4069

282/282 ━━━━━━━━━━━━━━━━━━━━ 48s 169ms/step - accuracy: 0.9365 - loss: 0.4069 - val_accuracy: 1.0000 - val_loss: 2.0901e-04
Epoch 5/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.9375 - loss: 0.3944

...

282/282 ━━━━━━━━━━━━━━━━━━━━ 83s 175ms/step - accuracy: 0.9659 - loss: 0.1643 - val_accuracy: 1.0000 - val_loss: 2.3881e-07
Epoch 44/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 81s 172ms/step - accuracy: 0.9672 - loss: 0.1582 - val_accuracy: 1.0000 - val_loss: 2.4395e-07
Epoch 45/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.9673 - loss: 0.1574

282/282 ━━━━━━━━━━━━━━━━━━━━ 82s 171ms/step - accuracy: 0.9673 - loss: 0.1574 - val_accuracy: 1.0000 - val_loss: 1.8055e-07
Epoch 46/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.9674 - loss: 0.1568

282/282 ━━━━━━━━━━━━━━━━━━━━ 49s 174ms/step - accuracy: 0.9674 - loss: 0.1568 - val_accuracy: 1.0000 - val_loss: 1.2142e-07
Epoch 47/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.9673 - loss: 0.1570

282/282 ━━━━━━━━━━━━━━━━━━━━ 82s 174ms/step - accuracy: 0.9673 - loss: 0.1570 - val_accuracy: 1.0000 - val_loss: 1.0502e-07
Epoch 48/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 81s 171ms/step - accuracy: 0.9688 - loss: 0.1506 - val_accuracy: 1.0000 - val_loss: 2.3293e-07
Epoch 49/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 48s 169ms/step - accuracy: 0.9689 - loss: 0.1496 - val_accuracy: 1.0000 - val_loss: 1.3205e-07
Epoch 50/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.9693 - loss: 0.1473

282/282 ━━━━━━━━━━━━━━━━━━━━ 83s 174ms/step - accuracy: 0.9693 - loss: 0.1473 - val_accuracy: 1.0000 - val_loss: 8.6595e-08


In [ ]:

# Cell 7.5 - Model Evaluation with BLEU Score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def translate_sentence(model, input_sequence, sp_amh, sp_eng, max_length=50):
    # Tokenize input
    amh_tokens = sp_amh.encode_as_ids(input_sequence)

    # Pad sequence
    # Note: For translation, we need to predict token by token,
    # so the input to the model's encoder needs to be the sequence.
    # The padding here is only for the encoder input shape.
    # However, the current model predicts the entire target sequence at once,
    # which is a typical setup for training but different for inference.
    # For this architecture's inference, a more standard approach is to
    # use the encoder's state and iteratively decode.
    # The current `translate_sentence` attempts a different approach by
    # feeding the predicted token back into the decoder input, which
    # requires a fixed-size input or dynamic shapes.
    # Let's adjust to use a more standard greedy decoding loop.

    # Encode the input sentence
    encoder_input_sequence = tf.constant([sp_amh.encode_as_ids(input_sequence)])

    # Get the initial state from the encoder
    # Need to access the encoder part of the model
    # Let's create an encoder model for inference
    encoder_model = Model(inputs=model.inputs[0], outputs=encoder_lstm.output[1:]) # assuming encoder_lstm outputs [output, h, c]


    state_h, state_c = encoder_model.predict(encoder_input_sequence, verbose=0)

    # Initialize decoder input with start token
    decoder_input_sequence = tf.constant([[2]])  # 2 is the start token (BOS)

    # Generate translation
    translated_tokens = []
    for _ in range(max_length):
        # Predict the next token
        # Need to access the decoder part of the model for inference
        # Let's create a decoder model for inference
        # This requires defining the decoder layers separately or
        # recreating the decoder model structure.
        # For simplicity, let's adapt the existing model.predict call,
        # but this structure is unusual for seq2seq inference.

        # Predict the next token using the full model, feeding the generated sequence
        # Note: This is not the standard way to do greedy decoding with this model structure.
        # A standard approach would be to have a separate decoder inference model
        # that takes the decoder input and encoder states, and outputs the next token probabilities
        # and updated states.
        # Let's proceed with the current structure's inference logic for now,
        # but acknowledge it's non-standard.

        output = model.predict([encoder_input_sequence, decoder_input_sequence], verbose=0)

        # Get the predicted token (take the last predicted token in the sequence)
        predicted_token = int(tf.argmax(output[0, -1, :]).numpy())

        # Break if end token is predicted or padding
        if predicted_token == 3 or predicted_token == 0:  # 3 is EOS, 0 is PAD
            break

        # Add predicted token to output
        translated_tokens.append(predicted_token)

        # Update decoder input for the next step
        # Append the predicted token to the decoder input sequence
        decoder_input_sequence = tf.concat([decoder_input_sequence, tf.constant([[predicted_token]])], axis=-1)


    # Convert tokens to text
    # Ensure tokens are ints for decoding
    translated_tokens = [int(t) for t in translated_tokens]
    try:
        english_text = sp_eng.decode(translated_tokens)
    except Exception as e:
        print(f"Error during decoding: {e}")
        # Fallback or handle error
        english_text = "" # Or return a placeholder

    return english_text

print("Evaluating model with BLEU score...")

# Get a sample of validation data for evaluation
num_samples = min(100, len(val_amh))  # Evaluate on up to 100 samples
sample_indices = np.random.choice(len(val_amh), num_samples, replace=False)

# Prepare reference translations and model predictions
references = []
hypotheses = []

for idx in sample_indices:
    # Get the Amharic sentence (original tokens before padding)
    # We need to get the original sentences or decode the padded ones and clean.
    # Let's decode the padded sequence and remove padding.
    amh_padded_tokens = val_amh[idx]
    # Filter out padding tokens (assuming pad_id is 0)
    amh_tokens = [token for token in amh_padded_tokens if token != 0]
    # Convert to text for the translate_sentence function
    amh_text = sp_amh.decode(amh_tokens)


    # Get the reference English translation (original tokens before padding)
    eng_padded_tokens = val_eng[idx]
     # Filter out padding tokens (assuming pad_id is 0) and BOS/EOS
    eng_tokens = [token for token in eng_padded_tokens if token not in [0, 2, 3]]
    eng_text = sp_eng.decode(eng_tokens)

    # Get model's translation
    predicted_text = translate_sentence(model, amh_text, sp_amh, sp_eng)

    # Tokenize for BLEU score
    # BLEU score typically uses space-tokenized words
    reference_tokens = [word_tokenize(eng_text.lower())]
    hypothesis_tokens = word_tokenize(predicted_text.lower())

    references.append(reference_tokens)
    hypotheses.append(hypothesis_tokens)

# Calculate BLEU score
bleu_score = corpus_bleu(references, hypotheses)
print(f"BLEU Score: {bleu_score:.4f}")

# Print some example translations
print("\nExample Translations:")
print("-" * 60)
for i in range(min(5, num_samples)):
    idx = sample_indices[i]
    # Decode padded sequences for display
    amh_padded_tokens = val_amh[idx]
    amh_tokens_display = [token for token in amh_padded_tokens if token != 0]
    amh_text_display = sp_amh.decode(amh_tokens_display)

    eng_padded_tokens = val_eng[idx]
    eng_tokens_display = [token for token in eng_padded_tokens if token not in [0, 2, 3]]
    eng_text_display = sp_eng.decode(eng_tokens_display)

    # Get the predicted translation using the function
    predicted_text_display = translate_sentence(model, amh_text_display, sp_amh, sp_eng)


    print(f"Amharic: {amh_text_display}")
    print(f"Reference: {eng_text_display}")
    print(f"Predicted: {predicted_text_display}")
    print("-" * 60)

# Save BLEU score to history
# Ensure history is a dictionary
if not hasattr(history, 'history'):
    history.history = {}
history.history['bleu_score'] = bleu_score

In [ ]:
# Save final model
print("Saving model...")
model.save("/content/drive/MyDrive/seq2seq_models/seq2seq_model_final.h5")

# Save training history
import json
with open("/content/drive/MyDrive/seq2seq_models/training_history.json", "w") as f:
    json.dump(history.history, f)

print("Training completed!")

# Create a zip file with all the necessary files
!zip -r /content/drive/MyDrive/seq2seq_models.zip /content/drive/MyDrive/seq2seq_models/

print("\nInstructions for downloading the models:")
print("1. Go to your Google Drive")
print("2. Find the seq2seq_models.zip file")
print("3. Download it")
print("4. Extract the contents to your local 'models' directory")

Saving model...
Training completed!
  adding: content/drive/MyDrive/seq2seq_models/ (stored 0%)
  adding: content/drive/MyDrive/seq2seq_models/tokenizer_amh.model (deflated 46%)
  adding: content/drive/MyDrive/seq2seq_models/tokenizer_amh.vocab (deflated 66%)
  adding: content/drive/MyDrive/seq2seq_models/tokenizer_eng.model (deflated 44%)
  adding: content/drive/MyDrive/seq2seq_models/tokenizer_eng.vocab (deflated 64%)
  adding: content/drive/MyDrive/seq2seq_models/seq2seq_model.h5 (deflated 7%)
  adding: content/drive/MyDrive/seq2seq_models/seq2seq_model_final.h5 (deflated 7%)
  adding: content/drive/MyDrive/seq2seq_models/training_history.json (deflated 56%)

Instructions for downloading the models:
1. Go to your Google Drive
2. Find the seq2seq_models.zip file
3. Download it
4. Extract the contents to your local 'models' directory
